In [1]:
'''Import de bibliotecas e opção de display de colunas'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
'''
Utilizar as seguintes tabelas: associacaoProdutoElo, blendagemProduto, clienteAPS, fornecedorAPS, patioAPS, 
previsaoAgregada, portoAPS, pesoMedioTransporteProduto, produtoAPS, submodalTransporte, trechoTransporte,
usinaBeneficiamentoAPS, usinaBriqueteAPS, usinaPelotizacaoAPS

'''

'\nUtilizar as seguintes tabelas: associacaoProdutoElo, blendagemProduto, clienteAPS, fornecedorAPS, patioAPS, \nprevisaoAgregada, portoAPS, pesoMedioTransporteProduto, produtoAPS, submodalTransporte, trechoTransporte,\nusinaBeneficiamentoAPS, usinaBriqueteAPS, usinaPelotizacaoAPS\n\n'

In [3]:
'''
Realiza a leitura dos dados em formato CSV.

Parâmetros:
- Caminhos dos arquivos CSV contendo os dados a serem lidos.

Retorno:
- DataFrames contendo os dados lidos do arquivo CSV.
'''
clientes_aps = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\clienteAPS.csv")
fornecedor_aps = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\fornecedorAPS.csv")
portos_aps = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\portoAPS.csv")
usina_beneficiamento_aps = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\usinaBeneficiamentoAPS.csv")
usinas_briquete_aps = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\usinaBriqueteAPS.csv")
usinas_pelotizacao_aps = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\usinaPelotizacaoAPS.csv")
patios_aps = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\patioAPS.csv")
trechos_transporte = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\trechoTransporte.csv")
peso_medio = pd.read_csv(r'C:\Users\pedro\Desktop\trataDados\csvs\pesoMedioTransporteProduto.csv')
produto_aps = pd.read_csv(r'C:\Users\pedro\Desktop\trataDados\csvs\produtoAPS.csv')
previ_agregada = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\previsaoAgregada.csv")
prod_elo = pd.read_csv(r"C:\Users\pedro\Desktop\trataDados\csvs\associacaoProdutoElo.csv")


In [4]:
'''
Reduz a tabela de produtoElo e remove valores NaN.

Parâmetros:
- prod_elo: DataFrame contendo os codigos dos produtos e idEloCadeiaProducao.

Retorno:
- DataFrame reduzido contendo apenas as colunas 'codigoProduto' e 'idEloCadeiaProducao' e sem valores NaN.
'''
prod_elo = prod_elo[["codigoProduto", "idEloCadeiaProducao"]]
prod_elo = prod_elo.dropna()


In [5]:
'''
Agrupa os produtos por idEloCadeiaProducao.

Parâmetros:
- prod_elo: DataFrame contendo os dados dos produtos e idEloCadeiaProducao.

Retorno:
- DataFrame agrupado por idEloCadeiaProducao, com a lista de códigos de produtos associados a cada id.
'''
prod_elo = prod_elo.groupby('idEloCadeiaProducao')['codigoProduto'].agg(list).reset_index()
prod_elo["id"] = prod_elo['idEloCadeiaProducao']
prod_elo = prod_elo.drop("idEloCadeiaProducao", axis=1)


In [6]:
import pandas as pd

def load_csv(filepath):
    """
    Carrega um arquivo CSV em um DataFrame do pandas.

    Parâmetros:
    - filepath: Caminho do arquivo CSV a ser carregado.

    Retorno:
    - DataFrame contendo os dados do arquivo CSV.
    """
    return pd.read_csv(filepath)

def preprocess_dataframes():
    """
    Pré-processa os dataframes individuais.

    Retorno:
    - Lista de dataframes contendo as colunas relevantes selecionadas.
    - Dataframes individuais para cada categoria: clientes_aps, fornecedor_aps, portos_aps, usina_beneficiamento_aps, usinas_briquete_aps, usinas_pelotizacao_aps, patios_aps.
    """
    dfs = [
        clientes_aps[["id", "descricao"]],
        fornecedor_aps[["id", "descricao"]],
        portos_aps[["id", "descricao", "capacidadeMaximaEstocagem"]],
        usina_beneficiamento_aps[["id", "descricao"]],
        usinas_briquete_aps[["id", "descricao"]],
        usinas_pelotizacao_aps[["id", "descricao"]],
        patios_aps[["id", "descricao", "capacidadeMaximaEstocagem"]],
    ]

    return dfs, clientes_aps, fornecedor_aps, portos_aps, usina_beneficiamento_aps, usinas_briquete_aps, usinas_pelotizacao_aps, patios_aps

def merge_and_categorize(dfs, clientes_aps, fornecedor_aps, portos_aps, usina_beneficiamento_aps, usinas_briquete_aps, usinas_pelotizacao_aps, patios_aps):
    """
    Mescla e categoriza os dataframes.

    Parâmetros:
    - dfs: Lista de dataframes contendo as colunas relevantes selecionadas.
    - Dataframes individuais para cada categoria: clientes_aps, fornecedor_aps, portos_aps, usina_beneficiamento_aps, usinas_briquete_aps, usinas_pelotizacao_aps, patios_aps.

    Retorno:
    - DataFrame resultante da mescla e categorização dos dataframes.
    """
    node_data = pd.concat(dfs).groupby('id', as_index=False).first()

    node_data["categorizacao"] = None
    categorization_conditions = [
        node_data['id'].isin(clientes_aps['id']),
        node_data['id'].isin(fornecedor_aps['id']),
        node_data['id'].isin(portos_aps['id']),
        node_data['id'].isin(usina_beneficiamento_aps['id']),
        node_data['id'].isin(usinas_briquete_aps['id']),
        node_data['id'].isin(usinas_pelotizacao_aps['id']),
        node_data['id'].isin(patios_aps['id']),
    ]

    for condition, category in zip(categorization_conditions, ["cliente", "fornecedor", "porto", "beneficiamento", "briquete", "pelotizacao", "patio"]):
        node_data.loc[condition, "categorizacao"] = category

    return node_data

def pipeline():
    """
    Executa o pipeline de pré-processamento.

    Retorno:
    - DataFrame final após o pré-processamento.
    """
    dfs, clientes_aps, fornecedor_aps, portos_aps, usina_beneficiamento_aps, usinas_briquete_aps, usinas_pelotizacao_aps, patios_aps = preprocess_dataframes()
    node_data = merge_and_categorize(dfs, clientes_aps, fornecedor_aps, portos_aps, usina_beneficiamento_aps, usinas_briquete_aps, usinas_pelotizacao_aps, patios_aps)
    return node_data

# Executa o pipeline
node_data = pipeline()

# Exibe o dataframe final
node_data = node_data.fillna(0)


In [7]:
"""
Realiza a junção entre o dataframe node_data e o dataframe prod_elo utilizando a coluna 'id' como chave de junção.

Parâmetros:
- node_data: DataFrame contendo os dados principais a serem combinados.
- prod_elo: DataFrame contendo os dados adicionais a serem adicionados ao node_data.

Retorno:
- DataFrame resultante da junção entre node_data e prod_elo.
"""
node_data = pd.merge(node_data, prod_elo, on="id")

In [8]:
'''
Seleciona as colunas necessárias do dataframe trechos_transporte.

Parâmetros:
- trechos_transporte: DataFrame contendo os dados dos trechos de transporte.

Retorno:
- DataFrame contendo apenas as colunas 'idEloCadeiaProducaoDestino', 'idEloCadeiaProducaoOrigem' e 'codigoSubModal'.
'''
trechos_transporte = trechos_transporte[['idEloCadeiaProducaoDestino', 'idEloCadeiaProducaoOrigem', 'codigoSubModal']]


In [9]:
'''
Realiza a categorização das rotas de transporte.

Parâmetros:
- trechos_transporte: DataFrame contendo os dados dos trechos de transporte.
- node_data: DataFrame contendo os dados dos nós.

Retorno:
- DataFrame categorizado das rotas de transporte.
'''
# Realiza o merge/junção para a categorização de origem
rotas_categorizadas = pd.merge(trechos_transporte, node_data, left_on='idEloCadeiaProducaoOrigem', right_on='id', how='left')
rotas_categorizadas.rename(columns={'categorizacao': 'categorizacaoOrigem', 'capacidadeMaximaEstocagem': 'capacidadeArmMaxOrigem', 'codigoProduto': 'prodOrigem'}, inplace=True)

# Realiza o merge/junção para a categorização de destino
rotas_categorizadas = pd.merge(rotas_categorizadas, node_data, left_on='idEloCadeiaProducaoDestino', right_on='id', how='left', suffixes=('', 'Destino'))
rotas_categorizadas.rename(columns={'categorizacao': 'categorizacaoDestino', 'capacidadeMaximaEstocagem': 'capacidadeArmMaxDestino', 'codigoProduto': 'prodDestino'}, inplace=True)

# Remove as colunas 'id' duplicadas que foram adicionadas durante o merge
rotas_categorizadas.drop(columns=['id', 'idDestino'], inplace=True)

rotas_categorizadas = rotas_categorizadas[["idEloCadeiaProducaoOrigem", "descricao", "categorizacaoOrigem", "capacidadeArmMaxOrigem", "prodOrigem", "idEloCadeiaProducaoDestino", "descricaoDestino", "categorizacaoDestino", "capacidadeArmMaxDestino", "prodDestino", "codigoSubModal"]]
rotas_categorizadas = rotas_categorizadas.dropna().reset_index(drop=True)

In [10]:
produto_desejado = "AF08"

In [11]:
'''
Seleciona as colunas necessárias do dataframe peso_medio e filtra os dados pelo produto desejado.

Parâmetros:
- peso_medio: DataFrame contendo os dados do peso médio de transporte por produto.
- produto_desejado: Código do produto desejado para filtragem.

Retorno:
- DataFrame contendo apenas as colunas 'codigoSubModal', 'pesoMedio' e 'codigoProduto' filtrado pelo produto desejado.
'''
peso_medio = peso_medio[["codigoSubModal", "pesoMedio", "codigoProduto"]] # SFHG SSFG AF08
peso_medio = peso_medio[peso_medio["codigoProduto"] == produto_desejado]

In [12]:
'''
Calcula a média dos pesos médios de transporte por produto e submodalidade.

Parâmetros:
- peso_medio: DataFrame contendo os dados do peso médio de transporte por produto e submodalidade.

Retorno:
- DataFrame contendo a média dos pesos médios de transporte por produto e submodalidade, arredondada para 2 casas decimais.
'''
media_pesos = peso_medio.groupby(['codigoProduto', 'codigoSubModal'])['pesoMedio'].mean().round(2).reset_index()

In [13]:
'''
Filtra as rotas categorizadas removendo aquelas que têm o produto desejado como origem.

Parâmetros:
- rotas_categorizadas: DataFrame contendo as rotas categorizadas.
- produto_desejado: Código do produto desejado para filtragem.

Retorno:
- DataFrame contendo as rotas categorizadas sem o produto desejado como origem.
'''
rotas_categorizadas = rotas_categorizadas[rotas_categorizadas["prodOrigem"] != produto_desejado]

In [14]:
'''
Realiza a junção dos dados das rotas categorizadas com as médias dos pesos médios de transporte por submodalidade.

Parâmetros:
- rotas_categorizadas: DataFrame contendo as rotas categorizadas.
- media_pesos: DataFrame contendo as médias dos pesos médios de transporte por produto e submodalidade.

Retorno:
- DataFrame contendo os dados tratados após a junção e remoção de linhas com valores nulos, reindexado.
'''
dados_tratados = pd.merge(rotas_categorizadas, media_pesos, on="codigoSubModal", suffixes=('_original', '_media'))
dados_tratados = dados_tratados.dropna().reset_index(drop=True)

In [15]:
'''
Filtra os dados tratados mantendo apenas as linhas em que o produto está presente tanto na origem quanto no destino.

Parâmetros:
- dados_tratados: DataFrame contendo os dados tratados após a junção.
- produto_desejado: Código do produto desejado para filtragem.

Retorno:
- DataFrame contendo apenas as linhas em que o produto está presente tanto na origem quanto no destino.
'''
df_filtrado = dados_tratados[dados_tratados.apply(lambda row: (row['codigoProduto'] in row['prodOrigem']) and (row['codigoProduto'] in row['prodDestino']), axis=1)]


In [16]:
'''
Filtra os dados de previsão agregada removendo as linhas em que a quantidade de produto é igual a zero.

Parâmetros:
- previ_agregada: DataFrame contendo os dados de previsão agregada.

Retorno:
- DataFrame contendo os dados de previsão agregada com quantidade de produto diferente de zero.
'''
previsao_agregada = previ_agregada[previ_agregada["qtdeProduto"] != 0]


In [17]:
'''
Filtra os dados de previsão agregada para demanda do tipo "F", identifica os períodos de maior frequência de início e fim,
e ajusta as colunas de data para formato datetime e adiciona colunas de dia de início e fim.

Parâmetros:
- previ_agregada: DataFrame contendo os dados de previsão agregada.

Retorno:
- DataFrame contendo os dados de previsão agregada filtrados para demanda do tipo "F" e ajustado com colunas de data e dia de início e fim.
'''
demanda_f = previ_agregada[previ_agregada["tipoDemanda"] == "F"]
high = demanda_f["dataInicio"].value_counts()
demanda_f = demanda_f[demanda_f["dataInicio"] == high.idxmax()]
high2 = demanda_f["dataFim"].value_counts()
demanda_f = demanda_f[demanda_f["dataFim"] == high2.idxmax()]
demanda_f['dataInicio'] = pd.to_datetime(demanda_f['dataInicio'])
demanda_f['dataFim'] = pd.to_datetime(demanda_f['dataFim'])
demanda_f['diaInicio'] = demanda_f['dataInicio'].dt.day
demanda_f['diaFim'] = demanda_f['dataFim'].dt.day
demanda_f = demanda_f.drop_duplicates(subset=['codigoElo', 'codigoProduto'], keep=False).reset_index(drop=True)
demanda_f = demanda_f[["codigoElo", "codigoProduto", "diaFim", "diaInicio", "qtdeProduto"]]


In [18]:
'''
Filtra os dados de previsão agregada para demanda do tipo "V", identifica os períodos de maior frequência de início e fim,
e ajusta as colunas de data para formato datetime e adiciona colunas de dia de início e fim.

Parâmetros:
- previ_agregada: DataFrame contendo os dados de previsão agregada.

Retorno:
- DataFrame contendo os dados de previsão agregada filtrados para demanda do tipo "V" e ajustado com colunas de data e dia de início e fim.
'''
demanda_v = previ_agregada[previ_agregada["tipoDemanda"] == "V"]
high = demanda_v["dataInicio"].value_counts()
demanda_v = demanda_v[demanda_v["dataInicio"] == high.idxmax()]
high2 = demanda_v["dataFim"].value_counts()
demanda_v = demanda_v[demanda_v["dataFim"] == high2.idxmax()]
demanda_v['dataInicio'] = pd.to_datetime(demanda_v['dataInicio'])
demanda_v['dataFim'] = pd.to_datetime(demanda_v['dataFim'])
demanda_v['diaInicio'] = demanda_v['dataInicio'].dt.day
demanda_v['diaFim'] = demanda_v['dataFim'].dt.day
demanda_v = demanda_v.drop_duplicates(subset=['codigoElo', 'codigoProduto'], keep=False).reset_index(drop=True)
demanda_v = demanda_v[["codigoElo", "codigoProduto", "diaFim", "diaInicio", "qtdeProduto"]]


In [19]:
'''
Concatena os DataFrames de demanda do tipo "F" e "V" e filtra os dados para o produto desejado.

Parâmetros:
- demanda_f: DataFrame contendo os dados de demanda do tipo "F".
- demanda_v: DataFrame contendo os dados de demanda do tipo "V".
- produto_desejado: Código do produto desejado para filtrar os dados.

Retorno:
- DataFrame resultante da concatenação das demandas e filtragem para o produto desejado.
'''
demandas = pd.concat([demanda_f, demanda_v]).reset_index(drop=True)
demandas = demandas[demandas["codigoProduto"] == produto_desejado]

In [20]:
"""
Realiza a junção (merge) de dois dataframes e aplica filtros aos dados resultantes.

Parâmetros:
- df_filtrado (pandas.DataFrame): DataFrame a ser mesclado à esquerda.
- demandas (pandas.DataFrame): DataFrame a ser mesclado à direita.
- produto_desejado (str): Código do produto desejado para filtrar os dados.

Retorno:
- pandas.DataFrame: DataFrame resultante da junção e aplicação de filtros.
"""
df_final = pd.merge(df_filtrado, demandas, left_on='idEloCadeiaProducaoDestino', right_on="codigoElo", how='left')
df_final.drop(["codigoProduto_x"], axis=1, inplace=True)
df_final = df_final[(df_final['categorizacaoDestino'] != 'cliente') | (df_final['qtdeProduto'].notna())]
df_final = df_final[df_final["qtdeProduto"] != 0]

In [21]:
"""
Remove colunas específicas do dataframe df_final e salva os dados restantes em um arquivo CSV.

Parâmetros:
- df_final (pandas.DataFrame): DataFrame contendo os dados a serem processados e salvos.
- colunas_para_remover (list): Lista de nomes das colunas a serem removidas.


Retorno:
- Dataframe com as colunas indicadas removidas.
"""
df_final = df_final.drop(["prodOrigem", "prodDestino", "codigoProduto_y", "codigoElo"], axis=1)


In [22]:
df_final.to_csv('dadosTratados.csv', index=False)

<h2>Teste de Neo4j</h2>

In [23]:
"""
Lê dados de um arquivo CSV tratado, cria consultas Cypher para um banco de dados de grafos e as escreve em um arquivo de texto.

Parâmetros:
- arquivo_csv (str): Nome do arquivo CSV tratado contendo os dados a serem processados.
- arquivo_cypher (str): Nome do arquivo de texto onde as consultas Cypher serão escritas.

Retorno:
- None
"""
import csv

# Abre o arquivo CSV para leitura
with open('dadosTratados.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    cypher_queries = []

    for row in reader:
        # Acessa os valores necessários da linha atual
        id_origem = row['idEloCadeiaProducaoOrigem']
        categorizacao_origem = row['categorizacaoOrigem']
        descricao_origem = row['descricao']
        id_destino = row['idEloCadeiaProducaoDestino']
        categorizacao_destino = row['categorizacaoDestino']
        descricao_destino = row['descricaoDestino']

        # Adiciona consulta Cypher para criar ou mesclar nó de origem
        cypher_queries.append(f"MERGE (o:{categorizacao_origem} {{id: '{id_origem}', descricao: '{descricao_origem}'}})")

        # Adiciona consulta Cypher para criar ou mesclar nó de destino
        cypher_queries.append(f"MERGE (d:{categorizacao_destino} {{id: '{id_destino}', descricao: '{descricao_destino}'}})")

        # Adiciona consulta Cypher para criar relação entre origem e destino
        cypher_queries.append(f"MATCH (o:{categorizacao_origem} {{id: '{id_origem}'}}), (d:{categorizacao_destino} {{id: '{id_destino}'}}) MERGE (o)-[:CONNECTION]->(d)")

# Escreve as consultas em um arquivo de texto
with open('cypher_queries.txt', 'w', encoding='utf-8') as file:
    for query in cypher_queries:
        file.write(query + ';\n')
